In [1]:
!mlflow --version

mlflow, version 2.14.2


Q1: mlflow, version 2.14.2

In [2]:
dest_path='homework_models'
raw_data_path='https://d37ci6vzurychx.cloudfront.net/trip-data/'

!mkdir -p $dest_path
!ls -l

total 28
-rw-rw-rw-  1 codespace codespace 7562 Jul 30 08:02 homework.ipynb
drwxrwxrwx+ 2 codespace codespace 4096 Jul 30 08:18 homework_models
-rw-rw-rw-  1 codespace codespace 1841 Jul 30 08:02 hpo.py
-rw-rw-rw-  1 codespace codespace 2510 Jul 30 08:02 preprocess_data.py
-rw-rw-rw-  1 codespace codespace 2502 Jul 30 08:02 register_model.py
-rw-rw-rw-  1 codespace codespace 1225 Jul 30 08:02 train.py


import preprocess_data
<br>- it isn't needed, because the function run_data_prep doesn't work with import because of 'click' context.
It would work if the function was splited to 2, the 1st for import, the 2nd for cli, like:

def run_data_prep(..):..

@click..

dev run_data_prep_cli(..): run_data_prep(..)`


In [3]:
# doesn't work because there's 'click' context
# preprocess_data.run_data_prep(raw_data_path=raw_data_path,dest_path=dest_path)
! python preprocess_data.py --raw_data_path="$raw_data_path" --dest_path "$dest_path"
# ! python preprocess_data.py --help

Q2: 4

so I took the liberty of changing train.py as it was mentioned above for preprocess_data.py, now it works both with import and with cli.

! python train.py --help

There's no need to wrap in with with mlflow.start_run(), mlflow.autolog() is triggered automaticly.

In [4]:
from train import run_train
# rt=run_train(dest_path)
rt=run_train(dest_path)
print(rt)

2024/07/30 08:18:58 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/07/30 08:18:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/07/30 08:19:00 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/07/30 08:19:00 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92

(RandomForestRegressor(max_depth=10, random_state=0), array([30.31046662, 21.65971103, 24.30442126, ..., 19.72563812,
       21.65971103, 14.82687034]), 5.431162180141208)


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


mlflow ui --backend-store-uri sqlite:///mlflow.db

Q3: 2

mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts_local

In [ ]:
# if callable(run_train): del run_train
server_uri='http://192.168.0.8:5000'


In [5]:
if callable(run_train): del run_train
from train import run_train
rt_s=run_train(dest_path,server_uri)
print(rt_s)

2024/07/30 08:19:20 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/07/30 08:19:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
WARNI [urllib3.connectionpool] Retrying (JitteredRetry(total=4, connect=4, read=5, redirect=5, status=5)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x77bdd4244b80>: Failed to establish a new connection: [Errno 111] Connection refused')': /api/2.0/mlflow/runs/create
WARNI [urllib3.connectionpool] Retrying (JitteredRetry(total=3, connect=3, read=5, redirect=5, status=5)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x77bdd42449d0>: Failed to establish a new connection: [Errno 111] Connection refused')': /api/2.0/mlflow/runs/create
WARNI [urllib3.connectionpool] Re

(RandomForestRegressor(max_depth=10, random_state=0), array([30.31046662, 21.65971103, 24.30442126, ..., 19.72563812,
       21.65971103, 14.82687034]), 5.431162180141208)


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Q4: default-artifact-root

In [7]:
import hpo

2024/07/30 08:56:38 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.


In [ ]:
# best_result = hpo.run_optimization(data_path='homework_models',num_trials=20, server_uri=server_uri, depth_from=20,depth_to=40)
best_result = hpo.run_optimization(data_path='homework_models',num_trials=10, server_uri=server_uri,experiment_prefix='withAproach')

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/07/31 14:03:43 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.

2024/07/31 14:03:44 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.

2024/07/31 14:03:45 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'

2024/07/31 14:04:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\halffiend\study\mlops-zoomcamp\.venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."



 10%|█         | 1/10 [00:31<04:39, 31.06s/trial, best loss: 5.341708058589234]

C:\Users\halffiend\study\mlops-zoomcamp\.venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/07/31 14:04:13 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.

2024/07/31 14:04:13 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.

2024/07/31 14:04:13 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'



 20%|██        | 2/10 [00:43<02:42, 20.35s/trial, best loss: 5.341708058589234]

...

 90%|█████████ | 9/10 [02:44<00:17, 17.71s/trial, best loss: 5.332842792557902]

C:\Users\halffiend\study\mlops-zoomcamp\.venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/07/31 14:06:26 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.

2024/07/31 14:06:26 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.

2024/07/31 14:06:27 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'



100%|██████████| 10/10 [03:00<00:00, 18.05s/trial, best loss: 5.332842792557902]

C:\Users\halffiend\study\mlops-zoomcamp\.venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



AllTrialsFailed: 

In [11]:
best_result

{'max_depth': 15.0,
 'min_samples_leaf': 2.0,
 'min_samples_split': 3.0,
 'n_estimators': 40.0}